In [1]:
import pandas as pd
import re

# CSVファイルの読み込み
df = pd.read_csv('suumo_scraping.csv')

# 各列の欠損値の数を確認
missing_values_count = df.isnull().sum()

# 「家賃」、「敷金」、「礼金」の「-」を「0」に置換し、不要な文字列を除去して数値に変換
columns_to_convert = ['家賃', '敷金', '礼金']
for column in columns_to_convert:
    df[column] = df[column].replace('-', '0')
    df[column] = df[column].replace('万円', '', regex=True)
    df[column] = pd.to_numeric(df[column], errors='coerce') * 10000

# 「管理費」列の「-」を「0」に置換し、不要な文字列を除去して数値に変換
df['管理費'] = df['管理費'].replace('-', '0')
df['管理費'] = df['管理費'].replace('円', '', regex=True)
df['管理費'] = pd.to_numeric(df['管理費'], errors='coerce')

# 「面積」列の単位を除去し、数値に変換
df['面積'] = df['面積'].replace('m2', '', regex=True)
df['面積'] = pd.to_numeric(df['面積'], errors='coerce')

# アクセス情報の処理と挿入
access_columns = ['アクセス1', 'アクセス2', 'アクセス3']
for col in access_columns:
    index = df.columns.get_loc(col)
    df[col + '_路線名'] = df[col].str.split('/', expand=True)[0]
    df[col + '_駅名'] = df[col].str.split('/', expand=True)[1].str.split(' 歩', expand=True)[0]
    df[col + '_徒歩分数'] = df[col].str.split('/', expand=True)[1].str.split(' 歩', expand=True)[1].str.replace('分', '', regex=True)
    df[col + '_徒歩分数'] = pd.to_numeric(df[col + '_徒歩分数'], errors='coerce')
    df.drop(col, axis=1, inplace=True)
    df.insert(index, col + '_路線名', df.pop(col + '_路線名'))
    df.insert(index + 1, col + '_駅名', df.pop(col + '_駅名'))
    df.insert(index + 2, col + '_徒歩分数', df.pop(col + '_徒歩分数'))

# 構造列を分割する関数
def split_structure(value):
    if '地下' in value:
        underground, aboveground = re.findall(r'\d+', value)
        return int(aboveground), int(underground)
    else:
        aboveground = re.search(r'\d+', value).group()
        return int(aboveground) if aboveground else 0, 0

# 構造列の位置を取得
structure_index = df.columns.get_loc('構造')

# 構造列を分割して新しい列を作成し、元の構造列の位置に挿入
df[['構造（地上）', '構造（地下）']] = df['構造'].apply(lambda x: pd.Series(split_structure(x)))
df.insert(structure_index, '構造（地上）', df.pop('構造（地上）'))
df.insert(structure_index + 1, '構造（地下）', df.pop('構造（地下）'))

# 元の構造列を削除
df.drop('構造', axis=1, inplace=True)

# 「階数」列の数値化と分割
def split_floors(floor_string):
    if pd.isna(floor_string):
        return None, None
    if '-' in floor_string:
        floors = floor_string.split('-')
        min_floor = re.search(r'\d+', floors[0])
        max_floor = re.search(r'\d+', floors[1])
        min_floor = int(min_floor.group()) if min_floor else None
        max_floor = int(max_floor.group()) if max_floor else None
        return min_floor, max_floor
    elif 'B' in floor_string or '地下' in floor_string:
        floor = re.search(r'\d+', floor_string)
        floor = int(floor.group()) * -1 if floor else None
        return None, floor
    else:
        floor = re.search(r'\d+', floor_string)
        floor = int(floor.group()) if floor else None
        return None, floor

df['最高階数'] = None
df['最低階数'] = None

for index, row in df.iterrows():
    min_floor, max_floor = split_floors(row['階数'])
    df.at[index, '最高階数'] = max_floor
    df.at[index, '最低階数'] = min_floor

# 「階数」列の位置を取得し、列を削除
floor_index = df.columns.get_loc('階数')
df.drop('階数', axis=1, inplace=True)

# 「最高階数」と「最低階数」の列を「構造（地下）」と「家賃」の間に移動
columns = list(df.columns)
columns.insert(columns.index('家賃'), columns.pop(columns.index('最高階数')))
columns.insert(columns.index('家賃'), columns.pop(columns.index('最低階数')))
df = df[columns]

# 「築年数」列の処理
df['築年数'] = df['築年数'].replace('新築', '0')  # 「新築」を「0」に置き換え
df['築年数'] = df['築年数'].str.extract('(\d+)').astype(float)  # 数字の抽出と数値化

# URL列の削除
df.drop('URL', axis=1, inplace=True)

# 処理後のデータフレームの先頭部分を表示して確認
print(df.head())

# すべての列が完全に一致する行の重複を削除
df = df.drop_duplicates()

# 処理後のデータフレームの先頭部分を表示して確認
print(df.head())

# CSVファイルに保存
df.to_csv('suumo_cleansing.csv', index=False, encoding='utf-8-sig')

      カテゴリ                  建物名               住所 アクセス1_路線名 アクセス1_駅名  \
0  賃貸マンション         ヴェルテＨＡＴＡＹＡＭＡ   神奈川県横浜市鶴見区栄町通４      京急本線    鶴見市場駅   
1  賃貸マンション  京急本線 鶴見市場駅 6階建 築18年   神奈川県横浜市鶴見区栄町通４      京急本線    鶴見市場駅   
2  賃貸マンション             ウノビスタ鶴見B  神奈川県横浜市鶴見区鶴見中央５   ＪＲ京浜東北線      鶴見駅   
3  賃貸マンション             ウノビスタ鶴見B  神奈川県横浜市鶴見区鶴見中央５   ＪＲ京浜東北線      鶴見駅   
4  賃貸マンション            ユアメゾン鶴見青色  神奈川県横浜市鶴見区鶴見中央４      京急本線    京急鶴見駅   

   アクセス1_徒歩分数 アクセス2_路線名 アクセス2_駅名  アクセス2_徒歩分数 アクセス3_路線名  ... 構造（地上）  構造（地下）  \
0         9.0     ＪＲ南武線    川崎新町駅        17.0     ＪＲ南武線  ...      6       0   
1         8.0   ＪＲ京浜東北線      鶴見駅        20.0     ＪＲ南武線  ...      6       0   
2         9.0      京急本線    京急鶴見駅         8.0     ＪＲ鶴見線  ...      4       0   
3         9.0      京急本線    京急鶴見駅         8.0     ＪＲ鶴見線  ...      4       0   
4         5.0   ＪＲ京浜東北線      鶴見駅         7.0      京急本線  ...      5       0   

   最高階数  最低階数        家賃   管理費        敷金        礼金   間取り     面積  
0     1  None   78000.0  6000   78000.0